In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet, Dataset, DataService
from pyincore.analyses.pipelinedamagerepairrate import PipelineDamageRepairRate
from pyincore.analyses.pipelinefunctionality import PipelineFunctionality

In [ ]:
# Connect to IN-CORE Service
client = IncoreClient()

In [ ]:
data_services = DataService(client)

### MMSA Pipiline damage 

In [ ]:
# Water Buried Pipeline inventory in Shelby county, TN
pipeline_dataset_id = "5a284f28c7d30d13bc081d14"
pipeline_dataset = Dataset.from_data_service(pipeline_dataset_id, data_services)

# pipeline fragility mapping
pipeline_mapping_id = "5b47c227337d4a38464efea8"

# New Madrid earthquake using Atkinson Boore 1995
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"

# Geology dataset
liq_geology_dataset_id = "5a284f53c7d30d13bc08249c"
liq_fragility_key = "pgd"
use_liq = True

In [ ]:
# Create pipeline damage with repair rate
pipeline_dmg_w_rr = PipelineDamageRepairRate(client)

# Load pipeline inventory as input datasets
pipeline_dmg_w_rr.load_remote_input_dataset("pipeline", pipeline_dataset_id)

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(pipeline_mapping_id))
pipeline_dmg_w_rr.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# Specify the result name
result_name = "pipeline_result"

# Set analysis parameters
pipeline_dmg_w_rr.set_parameter("result_name", result_name)
pipeline_dmg_w_rr.set_parameter("hazard_type", hazard_type)
pipeline_dmg_w_rr.set_parameter("hazard_id", hazard_id)
pipeline_dmg_w_rr.set_parameter("liquefaction_fragility_key", liq_fragility_key)
pipeline_dmg_w_rr.set_parameter("liquefaction_geology_dataset_id",liq_geology_dataset_id)
pipeline_dmg_w_rr.set_parameter("use_liquefaction", use_liq)
pipeline_dmg_w_rr.set_parameter("num_cpu", 4)

In [ ]:
# Run pipeline  damage analysis
pipeline_dmg_w_rr.run_analysis()

### Chaining with pipeline functionality

In [ ]:
pipeline_damage = pipeline_dmg_w_rr.get_output_dataset("result")

In [ ]:
pipline_func = PipelineFunctionality(client)

# Load input datasets
pipline_func.set_input_dataset("pipeline_repair_rate_damage", pipeline_damage)
# Load fragility mapping

# Set analysis parameters
pipline_func.set_parameter("result_name", "mmsa_pipeline_functionality")
pipline_func.set_parameter("num_samples", 100)

# Run pipeline analysis
pipline_func.run_analysis()

In [ ]:
pipline_func.get_output_dataset("failure_probability").get_dataframe_from_csv().head()

In [ ]:
pipline_func.get_output_dataset("sample_failure_state").get_dataframe_from_csv().head()